# C++ presentation for my beautiful daughter
## by Petru Marginean
### part 2


# Simple C++ app
<br>
Nothing special, just use `vi` to edit a simple `C++` program. The code prints at `stderr` the number of parameters, and at `stdout` the parameters.
```console
petrum@nuc ~/programming$ vi main.cpp

petrum@nuc ~/programming$ ls -ltr
total 4
-rw-rw-r-- 1 petrum petrum 267 Aug 20 08:21 main.cpp
petrum@nuc ~/programming$ cat main.cpp 
```

```cpp
#include <iostream>
 
int main(int argc, char* argv[])
{
    std::cerr << "We have received " << argc << " arguments.\n";
    for (int i = 0; i < argc; ++i)
    {
        std::cout << "- argv[" << i << "] = '" << argv[i] << "'" << std::endl;
    }
    return argc;
}
```

# Compilation + linking
<br>
Since we have a single compilation unit (main.cpp) we compile/link in a single step. We also introduce the file command, very handy to describe the file types (based on the file content)

```console
petrum@nuc ~/programming$ g++ main.cpp 

petrum@nuc ~/programming$ ls -ltr      
total 16
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:21 main.cpp
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 08:25 a.out

petrum@nuc ~/programming$ file main.cpp 
main.cpp: C source, ASCII text

petrum@nuc ~/programming$ file a.out
a.out: ELF 64-bit LSB executable, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, for GNU/Linux 2.6.32, BuildID[sha1]=123ba366434f809a5c2a429905b2c3f0a0243489, not stripped
```

# Running the program
<br>
We run the program twice: we first give no parameters (and surprinsingly notice the `argc == 1`!) ans second we pass some random parameters (strings separated by spaces)
```console
petrum@nuc ~/programming$ ./a.out 
We have received 1 arguments.
- argv[0] = './a.out'

petrum@nuc ~/programming$ ./a.out Hello World!
We have received 3 arguments.
- argv[0] = './a.out'
- argv[1] = 'Hello'
- argv[2] = 'World!'
```

# Pipes, parameters and return code
<br>
We use in general the command line parameters to parameterize the app, the `stdin` to provide the data we work on, `stdout` to output dthe generated data (buffered), the `stderr` to output the error messages (unbuffered, fast display), and the error code to signal success / errors to the caller.

````

                                  | command line
                                  |  parameters
                                  | (argc/argv)
                                  v
                +---------------------------------+
                |                                 |
                |                                 |
    stdin       |                                 |   stdout
  ==========>   |           application           | ===========>
   std:cin      |(C/C++, Python, Ruby, Perl etc)  |  std::cout
                |                                 | (buffered)
                +---------------------------------+
                          |            |
                          |            |
                  stderr  |            | return code
                 std:cerr |            |
              (unbuffered)|            |
                          v            v


```

# Running - stdout redirection
<br>
We introduce the putput redirection: by using the `>` redirection operator we redirect the `stdout` from the terminal into a file!

```console
petrum@nuc ~/programming$ ls -ltr
total 16
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:21 main.cpp
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 08:25 a.out

petrum@nuc ~/programming$ ./a.out Hello World! >out.std
We have received 3 arguments.

petrum@nuc ~/programming$ ls -ltr
total 20
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:21 main.cpp
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 08:25 a.out
-rw-rw-r-- 1 petrum petrum   63 Aug 20 08:26 out.std

petrum@nuc ~/programming$ cat out.std 
- argv[0] = './a.out'
- argv[1] = 'Hello'
- argv[2] = 'World!'
```

# Running - full redirection
<br>
Here we redirect both stdout and stderr. The app prints nothing on the screen anymore. Both stdout and stderr do into different files. The `stdout` has the `1` file handler (or the default) and `stderr` has the `2` file handler.
```console
petrum@nuc ~/programming$ ./a.out Hello World! 2>err.std >out.std

petrum@nuc ~/programming$ ls -ltr
total 24
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:21 main.cpp
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 08:25 a.out
-rw-rw-r-- 1 petrum petrum   63 Aug 20 08:26 out.std
-rw-rw-r-- 1 petrum petrum   30 Aug 20 08:26 err.std

petrum@nuc ~/programming$ cat err.std 
We have received 3 arguments.
```

# Running with appending
<br>
We use the `>>` operator to append to a file (instead of overwriting it)
```console
petrum@nuc ~/programming$ ./a.out Hello World! 2>>err.std >>out.std

petrum@nuc ~/programming$ ls -ltr
total 24
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:21 main.cpp
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 08:25 a.out
-rw-rw-r-- 1 petrum petrum  126 Aug 20 08:27 out.std
-rw-rw-r-- 1 petrum petrum   60 Aug 20 08:27 err.std

petrum@nuc ~/programming$ cat err.std 
We have received 3 arguments.
We have received 3 arguments.
```

# Running – error code
<br>
Id we are not interested in reading some output (and we want to discard it) we can use `/dev/null` as destination. We also can read the reurn code the app returns (equal to the number of parameters, remember `return argc;`?)

Any Linux command returns an error code! 

For example below the `ls` command return `0` (no error) if everything is fine or `2` if the listed fine is not available.
```console
petrum@nuc ~/programming$ ./a.out Hello World! >/dev/null
We have received 3 arguments.

petrum@nuc ~/programming$ echo $?
3

petrum@nuc ~/programming$ ls
a.out  err.std  main.cpp  out.std

petrum@nuc ~/programming$ echo $?
0

petrum@nuc ~/programming$ ls nothing
ls: cannot access 'nothing': No such file or directory

petrum@nuc ~/programming$ echo $?
2
```

# More ‘file’ output
<br>
Using the `*` and running the `file` comand with multiple files in a single shot.
```console
petrum@nuc ~/programming$ file *
a.out:    ELF 64-bit LSB executable, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, for GNU/Linux 2.6.32, BuildID[sha1]=123ba366434f809a5c2a429905b2c3f0a0243489, not stripped
err.std:  ASCII text
main.cpp: C source, ASCII text
out.std:  ASCII text
```

# Compilation only
<br>
Instead of doing compilation + linking in a single shot we only do the compile part (using the `-c` parameter). A single object file is generated from the compilation unit (the same filename, but with the `*.o` extension)
```console
petrum@nuc ~/programming$ g++ -c main.cpp

petrum@nuc ~/programming$ echo $?
0

petrum@nuc ~/programming$ ls -ltr
total 28
-rw-rw-r-- 1 petrum petrum  126 Aug 20 08:27 out.std
-rw-rw-r-- 1 petrum petrum   60 Aug 20 08:27 err.std
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 08:38 app.exe
-rw-rw-r-- 1 petrum petrum  266 Aug 20 08:40 main.cpp
-rw-rw-r-- 1 petrum petrum 3224 Aug 20 08:40 main.o

petrum@nuc ~/programming$ file main.o 
main.o: ELF 64-bit LSB relocatable, x86-64, version 1 (SYSV), not stripped
```

# Compilation + linking
<br>
After compilation only we can link together multiple `*.o` and libraries. Please note in the linking phase we do not use the source code anymore (not `*.cpp`, only `*.o` and eventually `*.a` or `*.so`)
```console
petrum@nuc ~/programming$ ls -ltr
total 4
-rw-rw-r-- 1 petrum petrum 267 Aug 20 08:51 main.cpp

petrum@nuc ~/programming$ g++ -c main.cpp

petrum@nuc ~/programming$ ls -ltr
total 8
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:51 main.cpp
-rw-rw-r-- 1 petrum petrum 3224 Aug 20 09:04 main.o

petrum@nuc ~/programming$ g++ main.o

petrum@nuc ~/programming$ ls -ltr
total 20
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:51 main.cpp
-rw-rw-r-- 1 petrum petrum 3224 Aug 20 09:04 main.o
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 09:04 a.out
```

# Specifying output name (‘-o name’)
<br>
Instead of the defualt `a.out` out file name we can specify the out filename using the `-o` parameter. of course we could use the `mv` command to get a similar effect.
```console
petrum@nuc ~/programming$ g++ main.o -o app.exe

petrum@nuc ~/programming$ ls -ltr
total 32
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:51 main.cpp
-rw-rw-r-- 1 petrum petrum 3224 Aug 20 09:04 main.o
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 09:04 a.out
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 09:06 app.exe

petrum@nuc ~/programming$ mv a.out app.exe 

petrum@nuc ~/programming$ ls -ltr
total 20
-rw-rw-r-- 1 petrum petrum  267 Aug 20 08:51 main.cpp
-rw-rw-r-- 1 petrum petrum 3224 Aug 20 09:04 main.o
-rwxrwxr-x 1 petrum petrum 9320 Aug 20 09:04 app.exe
```

# The end!
## Good luck to U-Mich!!!
### With love daddy!
